In [3]:
using Pkg

In [4]:
Pkg.add(["DataFrames", "CSV", "Query", "StatsBase"])
using DataFrames, CSV, Query, StatsBase

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [31]:
ENV["COLUMNS"] = 1000; ENV["LINES"] = 20

20

In [32]:
df_horizon_orig = CSV.read("WISE3_HORIZON.csv", DataFrame)

┌ Warning: thread = 1 warning: only found 30 / 31 columns around data row: 47834. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603


,WISE3_ID,HONU,DESIG,TOPDEP,BOTDEP,MCOLOR,DCOLOR,ORGC,TOTN,CACO3,GYPSUM,PHH2O,PHKCL,PHCACL2,ECE,EXCA,EXMG,EXNA,EXK,EXALUM,EXACID,CECSOIL,BSAT,SAND,SILT,CLAY,GRAVEL,BULKDENS,VMC1,VMC2,VMC3
,String,Int64,String?,Int64,Int64,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,Int64?,Int64?,Int64?,Int64?,Int64?,String?,Int64?,Int64?,Int64?
1,AF0001,1,missing,0,15,10YR4/3,10YR4/2,"7,6","0,6",93,missing,"7,9",missing,missing,"0,38",missing,missing,missing,missing,0,0,missing,missing,40,40,20,20,missing,missing,missing,missing
2,AF0001,2,missing,15,60,10YR5/3,10YR5/2,"2,3","0,3",177,missing,"7,9",missing,missing,"0,3",missing,missing,missing,missing,0,0,missing,missing,10,55,35,missing,missing,missing,missing,missing
3,AF0001,3,missing,60,150,10YR5/3,missing,"0,9","0,3",182,missing,"7,9",missing,missing,"0,28",missing,missing,missing,missing,0,0,missing,missing,10,55,35,missing,missing,missing,missing,missing
4,AF0002,1,missing,0,20,10YR4/2,missing,"12,8","0,9",191,missing,"8,5",missing,missing,"0,42",missing,missing,missing,missing,0,0,missing,missing,40,40,20,missing,missing,missing,missing,missing
5,AF0002,2,missing,20,60,10YR4/2,missing,6,"0,6",192,missing,"8,6",missing,missing,"0,34",missing,missing,missing,missing,0,0,missing,missing,15,65,20,missing,missing,missing,missing,missing
6,AF0002,3,missing,60,110,10YR4/3,missing,"3,9","0,6",203,missing,"8,5",missing,missing,"0,3",missing,missing,missing,missing,0,0,missing,missing,10,55,35,missing,missing,missing,missing,missing
7,AF0002,4,missing,110,170,10YR4/3,missing,"2,7",missing,240,missing,"8,8",missing,missing,"0,3",missing,missing,missing,missing,0,0,missing,missing,40,40,20,missing,missing,missing,missing,missing
8,AF0003,1,A,0,20,10YR4/2,missing,"5,9","0,7",169,missing,"8,8",missing,missing,"1,9",missing,missing,missing,missing,0,0,missing,missing,40,40,20,missing,missing,missing,missing,missing
9,AF0003,2,Bg,20,50,10YR4/2,missing,"2,4","0,4",138,missing,"9,2",missing,missing,"0,55",missing,missing,missing,missing,0,0,missing,missing,65,25,10,missing,missing,missing,missing,missing


In [6]:
function tidy_names(old_names)

    return new_names = old_names |>
    #remove leading and trailing spaces
    n -> strip.(n) |>

    #replace spaces with underscores
    n -> replace.(n, ' ' => '_') |> 

    #remove parenthesis
    n -> replace.(n,'(' => "") |> 
    n -> replace.(n,')' => "") |>

    #remove dashes and dots
    n -> replace.(n,'-' => "") |> 
    n -> replace.(n, '.' => "") |>

    #all lowercase
    n -> lowercase.(n)

end

tidy_names (generic function with 1 method)

In [7]:
function infer_designation(old_desig, blk)
#takes layer and returns its inferred soil designation, either 'organic' or 'mineral'
    new_desig = "no_desig" #value to be eventually returned

    threshold = 0.5

    control_desig = ['O','o','H','h'] #possible chars in existing desig that indicate presence of organic material in the layer

    new_desig_organic = "organic"
    new_desig_mineral = "mineral"

    #if designation data is missing 
    if old_desig == ""
    #check on bulk density value to decide which designation applies 
        new_desig = (blk < threshold) ? new_desig_organic : new_desig_mineral

    #designation not missing
    else
        check_desig = [in(c,control_desig) for c in old_desig] #check if any char in the existing designation matches organic control
        check_desig_BD = !isnothing(findfirst(check_desig)) & (blk < threshold) #true if any char matches and BD < treshold

        new_desig = check_desig_BD ? new_desig_organic : new_desig_mineral
    end


    return new_desig
end

infer_designation (generic function with 1 method)

In [42]:
#general clean-up	
df_hz = df_horizon_orig |> 
#pick only relevant columns	
@select(:WISE3_ID,:HONU,:DESIG,:TOPDEP,:BOTDEP,:ORGC,:BULKDENS) |>

#drop layers we can't have missing values for
@dropna(:BULKDENS,:ORGC,:TOPDEP,:BOTDEP) |>

#includes only layers with bot_depth greater than top_depth
@filter(_.BOTDEP > _.TOPDEP) |> 

#parse ORGC values and keep only non-negative ones
@mutate(ORGC = replace(_.ORGC,','=>'.')) |>
@mutate(ORGC = parse(Float64, _.ORGC)) |>
@filter(_.ORGC >= 0) |>

#parse BULKDENS values and keep only positive ones
@mutate(BULKDENS = replace(_.BULKDENS,','=>'.')) |>
@mutate(BULKDENS = parse(Float64, _.BULKDENS)) |>
@filter(_.BULKDENS > 0) |> 

#add column with inferred layer designation
@mutate(infer_DESIG = infer_designation(get(_.DESIG,""),_.BULKDENS)) |>	

#keep only unique layers
@unique() |>


DataFrame

,WISE3_ID,HONU,DESIG,TOPDEP,BOTDEP,ORGC,BULKDENS,infer_DESIG
,String,Int64,String?,Int64,Int64,Float64,Float64,String
1,AL0007,1,Ap,0,25,9.8,1.64,mineral
2,AL0007,2,AB,25,49,7.8,1.62,mineral
3,AL0007,3,Bw1,49,83,5.9,1.57,mineral
4,AL0007,4,Bw2,83,116,6.9,1.63,mineral
5,AL0007,5,2C1,116,147,4.6,1.56,mineral
6,AL0007,6,2C2,147,160,4.6,1.45,mineral
7,AL0008,1,Ap-,0,17,11.5,1.61,mineral
8,AL0008,2,Bw1,17,38,8.1,1.55,mineral
9,AL0008,3,2Bw2,38,66,1.7,1.18,mineral


In [9]:
function continuity_check(profile) 
#the function assumes layers are ordered (e.g.  0 -> 20 -> 40)

    top = profile.TOPDEP
    bot = profile.BOTDEP

    num_layers = length(top)

    #1st layer is always good
    mask = [true]

    #from 2nd layer downward, we check for matching depths at bot/top
    mask = append!(mask, [top[l] == bot[l-1] for l in 2:num_layers])

    #if we run into a non-matching layer, we mark as 'false' all following ones
    first_noMatch_id = findfirst(!, mask)

    if !isnothing(first_noMatch_id) #true if a non-contiguous layer has been found
        mask[first_noMatch_id:end] .= false
    end

    return mask	
end

continuity_check (generic function with 1 method)

In [10]:
function continuity_check_df(df)
#generate continuity mask for the whole dataframe
continuous_layers = Bool[]

for profile in unique(df.WISE3_ID)
    append!(continuous_layers, continuity_check(df[df.WISE3_ID.==profile,:]))
end

return continuous_layers

end

continuity_check_df (generic function with 1 method)

In [11]:
df_hz_cont = df_hz[continuity_check_df(df_hz),:]; #df with only continuous profiles

In [12]:
function depth_clip(profile; depth_min=0, depth_max=100)
#takes in a profile with N layers and returns a clipped profile, by default from 0 to 100cm
#BD and ORGC are left unchanged, as they're concentration measures and don't depend on layers' height
#run this function ALWAYS AFTER checking for layers continuity

top = profile.TOPDEP
bot = deepcopy(profile.BOTDEP) #create copy of the original profile bot, to be edited later

n_layers = length(top)

#init array of 'true' -> keep all layers unless proven guilty
layers_keep = Vector{Bool}(undef,n_layers)
layers_keep .= true

#if profile doesn't reach threshold, or starts after min, ditch the whole profile
if maximum(bot) < depth_max || minimum(top) > depth_min; 	
    layers_keep .= false
        
#othewise mark layers to drop above threshold, and clip the overlapping one
else 
    for lr in 1:n_layers
        #simple case, both top and bot below threshold
        if top[lr] >= depth_max; layers_keep[lr] = false; end

        #layers overlapping threshold
        if top[lr] < depth_max && bot[lr] > depth_max
            bot[lr] = depth_max
        end

    end

    profile.BOTDEP = bot
end

return profile[layers_keep,:]
end

depth_clip (generic function with 1 method)

In [13]:
function compute_clipped_profiles(df)
#initialize empty DF with same columns and types as starting one
results = DataFrame(eltypes(df), names(df))

for profile in unique(df.WISE3_ID)
    append!(results, depth_clip(df[df.WISE3_ID .== profile,:]))
end

return results

end

compute_clipped_profiles (generic function with 1 method)

In [14]:
#example of depth_clip in action:
test_profile = df_hz_cont[df_hz_cont.WISE3_ID .== "AL0007",:]

,WISE3_ID,HONU,DESIG,TOPDEP,BOTDEP,ORGC,BULKDENS,infer_DESIG
,String,Int64,String?,Int64,Int64,Float64,Float64,String
1,AL0007,1,Ap,0,25,9.8,1.64,mineral
2,AL0007,2,AB,25,49,7.8,1.62,mineral
3,AL0007,3,Bw1,49,83,5.9,1.57,mineral
4,AL0007,4,Bw2,83,116,6.9,1.63,mineral
5,AL0007,5,2C1,116,147,4.6,1.56,mineral
6,AL0007,6,2C2,147,160,4.6,1.45,mineral


In [15]:
depth_clip(test_profile)

,WISE3_ID,HONU,DESIG,TOPDEP,BOTDEP,ORGC,BULKDENS,infer_DESIG
,String,Int64,String?,Int64,Int64,Float64,Float64,String
1,AL0007,1,Ap,0,25,9.8,1.64,mineral
2,AL0007,2,AB,25,49,7.8,1.62,mineral
3,AL0007,3,Bw1,49,83,5.9,1.57,mineral
4,AL0007,4,Bw2,83,100,6.9,1.63,mineral


In [34]:
df_hz_clipped = compute_clipped_profiles(df_hz_cont)

,WISE3_ID,HONU,DESIG,TOPDEP,BOTDEP,ORGC,BULKDENS,infer_DESIG
,String,Int64,String?,Int64,Int64,Float64,Float64,String
1,AL0007,1,Ap,0,25,9.8,1.64,mineral
2,AL0007,2,AB,25,49,7.8,1.62,mineral
3,AL0007,3,Bw1,49,83,5.9,1.57,mineral
4,AL0007,4,Bw2,83,100,6.9,1.63,mineral
5,AL0008,1,Ap-,0,17,11.5,1.61,mineral
6,AL0008,2,Bw1,17,38,8.1,1.55,mineral
7,AL0008,3,2Bw2,38,66,1.7,1.18,mineral
8,AL0008,4,2Bw3,66,100,1.5,0.89,mineral
9,AL0010,1,Ap,0,17,18.1,1.52,mineral


In [17]:
function orgc_storage_desig(profile) 

    id = profile.WISE3_ID[1]
    top = profile.TOPDEP
    bot = profile.BOTDEP
    des = profile.infer_DESIG

    blk = profile.BULKDENS
    orgc = profile.ORGC

    layer_height = (bot.-top)
    num_layer = length(top)

    orgc_stored_mineral = 0.0
    orgc_stored_organic = 0.0

    for l in 1:num_layer
        if des[l] == "organic"; orgc_stored_organic += blk[l]*orgc[l]*layer_height[l]/100; end
        if des[l] == "mineral"; orgc_stored_mineral += blk[l]*orgc[l]*layer_height[l]/100; end
    end

    return DataFrame(profile_id = id, 
                     orgc_stored_mineral = orgc_stored_mineral, 
                     orgc_stored_organic = orgc_stored_organic)
end

orgc_storage_desig (generic function with 1 method)

In [18]:
function compute_orgc_storage_desig(df)

    unique_profiles = unique(df.WISE3_ID)
    #initialize result var with 1st call to function
    result = orgc_storage_desig(df[df.WISE3_ID .== unique_profiles[1],:])

    for profile in unique_profiles[2:end]
        append!(result, orgc_storage_desig(df[df.WISE3_ID .== profile,:]))
    end

    return result
end

compute_orgc_storage_desig (generic function with 1 method)

In [35]:
#compute stored ORGC for continuous, not clipped, profiles
df_storage_cont = compute_orgc_storage_desig(df_hz_cont)

,profile_id,orgc_stored_mineral,orgc_stored_organic
,String,Float64,Float64
1,AL0007,17.0032,0.0
2,AL0008,6.79968,0.0
3,AL0009,10.19,0.0
4,AL0010,16.182,0.0
5,AL0011,19.5227,0.0
6,AL0012,12.7518,0.0
7,AL0013,14.3507,0.0
8,AL0014,13.6746,0.0
9,AL0015,23.7756,0.0


In [36]:
#compute stored ORGC for continuous and clipped profiles
df_storage_clipped = compute_orgc_storage_desig(df_hz_clipped)

,profile_id,orgc_stored_mineral,orgc_stored_organic
,String,Float64,Float64
1,AL0007,12.112,0.0
2,AL0008,6.79968,0.0
3,AL0010,15.782,0.0
4,AL0011,17.5067,0.0
5,AL0012,7.55993,0.0
6,AL0013,10.7694,0.0
7,AL0014,13.0122,0.0
8,AL0015,20.5962,0.0
9,AL0016,103.021,0.0


In [37]:
df_site_orig = CSV.read("WISE3_SITE.csv", DataFrame)

,WISE3_id,ISO,DESCR,DATEYR,DATEMON,SOLDEP,HORNUM,FAO_90,PHA_90,FAO_74,PHA_74,WRB2006,USCL,USYR,LOCAL,LONDD,LATDD,LONLAT_ACC,LOCAT,KOPPEN,ALTIT,LFORM,POSIT,SLOPE,PARMAT,DRAIN,LANDUS,SOURCE_ID,PITREF,LAB_ID,COUNTRY
,String,String,Int64,Int64?,Int64?,Int64,Int64,String,String?,String?,String?,String,String?,Int64?,String?,String?,String?,String?,String?,String?,Int64?,String?,String?,String?,String?,String,String?,String,String,String,String
1,AF0001,AF,3,1962,missing,150,3,CLl,missing,Bk,missing,CL,missing,missing,Medium textured upland soil,"69,16667","34,5",M,Kabul valley.,Dsb,1800,LV,LS,missing,UC,W,AA4,AS3/73,Profile 16 p10-11.,AF01,Afghanistan
2,AF0002,AF,3,1962,missing,170,4,FLc,missing,Jc,missing,FL,missing,missing,Alluvial soil with thick irrigation horizon,"69,16667","34,5",M,Logar valley S of Kabul,Dsb,1800,LV,LS,missing,UF,M,AA6,AS3/73,"Profile 20, pp 15-16.",AF01,Afghanistan
3,AF0003,AF,3,1962,missing,110,3,FLc,missing,Jc,missing,FL,missing,missing,Grey-brown hydromorphic soil,"69,16667","34,5",M,Kabul valley,Dsb,1800,LV,LO,missing,UF,P,AA6,AS3/73,"Profile 41, pp 20-21.",AF01,Afghanistan
4,AF0004,AF,3,1962,missing,270,6,SCg,missing,Zo,missing,SC,missing,missing,Grey-brown hydromorphic saline soil,"61,43333","34,33333",M,"Ghurian basin, west of Herat.",BSk,930,LV,BO,missing,UF,M,AA4,AS3/73,"Profile 17, pp 96-97.",AF01,Afghanistan
5,AF0005,AF,3,1962,missing,190,6,SCk,missing,Zo,missing,SC,missing,missing,Solonchak,"62,13333","32,38333",M,"Farah valley, western Afganistan",BWh,650,LV,LO,missing,UF,P,HE2,AS3/73,"Profile 21, pp 74-75.",AF01,Afghanistan
6,AL0001,AL,4,1981,9,94,4,LVk,missing,Lk,missing,LV,missing,missing,Marroon Meadow Soils,"20,75","40,61667",M,Drenova,Dfb,600,SH,BO,5,SC2,W,AA4,EU14/AISS,"Soil Institute, Tirana",AL01,Albania
7,AL0002,AL,4,1983,9,87,3,LPk,missing,Be,missing,LP,missing,missing,Marroon grey soil,"20,30056","39,635",S,Konispol,Csa,250,SH,UP,15,SO1,W,AA2,EU10/AISS,"Institute of Soil Science, Tirana",AL01,Albania
8,AL0003,AL,4,1985,12,85,3,LVf,missing,Lf,missing,LV,missing,missing,Marroon meadow soil,"20,38611","42,22222",S,Pla-Kruma,Dfb,565,SM,LS,12,SC2,W,AA1,EU10/AISS,"Institute of Soil Science, Tirana",AL01,Albania
9,AL0004,AL,4,1991,11,120,6,CMc,missing,Bk,missing,CM,missing,missing,Marroon Grey Soil,"19,56111","41,07778",S,Kavaja Rogozhina,Csa,33,SH,UP,60,SC1,W,AT1,EU14/AISS,"Institute of Soil Science, Tirana.",AL01,Albania


In [43]:
df_site = df_site_orig |>
    @select(:WISE3_id, :DATEYR, :LATDD, :LONDD) |>

    @replacena(:LONDD=>"-300") |>
    @replacena(:LATDD=>"-300") |>

    #parse LONDD values
    @mutate(LONDD = replace(_.LONDD,','=>'.')) |>
    @mutate(LONDD = parse(Float64, _.LONDD)) |>

    #parse LATDD values
    @mutate(LATDD = replace(_.LATDD,','=>'.')) |>
    @mutate(LATDD = parse(Float64, _.LATDD)) |>

    @rename(:WISE3_id => :profile_id) |>

    DataFrame

,profile_id,DATEYR,LATDD,LONDD
,String,Int64?,Float64,Float64
1,AF0001,1962,34.5,69.1667
2,AF0002,1962,34.5,69.1667
3,AF0003,1962,34.5,69.1667
4,AF0004,1962,34.3333,61.4333
5,AF0005,1962,32.3833,62.1333
6,AL0001,1981,40.6167,20.75
7,AL0002,1983,39.635,20.3006
8,AL0003,1985,42.2222,20.3861
9,AL0004,1991,41.0778,19.5611


In [23]:
function depth_desig(profile) 

    id = profile.WISE3_ID[1]
    des = profile.infer_DESIG
    top = profile.TOPDEP
    bot = profile.BOTDEP

    missing_val = -1

    num_layers = length(top)

    mineral_index = [i for i in 1:num_layers if des[i] == "mineral"]
    organic_index = [i for i in 1:num_layers if des[i] == "organic"]

    #values to return

    profile_top = minimum(top)
    profile_bot = maximum(bot)

    mineral_top = isempty(mineral_index) ? missing_val : minimum(top[mineral_index])
    mineral_bot = isempty(mineral_index) ? missing_val : maximum(bot[mineral_index])

    organic_top = isempty(organic_index) ? missing_val : minimum(top[organic_index])
    organic_bot = isempty(organic_index) ? missing_val : maximum(bot[organic_index])


    return DataFrame(profile_id = id, 
                     profile_top = profile_top,
                     profile_bot = profile_bot,
                     organic_top = organic_top,
                     organic_bot = organic_bot,
                     mineral_top = mineral_top,
                     mineral_bot = mineral_bot)

end

depth_desig (generic function with 1 method)

In [24]:
function compute_depth_desig(df)

    unique_profiles = unique(df.WISE3_ID)

    #initialize results with 1st call to function
    result = depth_desig(df[df.WISE3_ID .== unique_profiles[1],:])

    for profile in unique_profiles[2:end]
        append!(result, depth_desig(df[df.WISE3_ID .== profile,:]))
    end

    return result
end

compute_depth_desig (generic function with 1 method)

In [39]:
#designations' depths per profile (non clipped)
df_cont_depth_desig = compute_depth_desig(df_hz_cont)

,profile_id,profile_top,profile_bot,organic_top,organic_bot,mineral_top,mineral_bot
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,AL0007,0,160,-1,-1,0,160
2,AL0008,0,100,-1,-1,0,100
3,AL0009,0,97,-1,-1,0,97
4,AL0010,0,110,-1,-1,0,110
5,AL0011,0,130,-1,-1,0,130
6,AL0012,0,173,-1,-1,0,173
7,AL0013,0,180,-1,-1,0,180
8,AL0014,0,130,-1,-1,0,130
9,AL0015,0,180,-1,-1,0,180


In [26]:
#designations depths per clipped profiles 
df_clipped_depth_desig = compute_depth_desig(df_hz_clipped);

In [40]:
#joining results with extra info	
df_cont_results = innerjoin(df_storage_cont, df_cont_depth_desig, df_site, on = :profile_id)

,profile_id,orgc_stored_mineral,orgc_stored_organic,profile_top,profile_bot,organic_top,organic_bot,mineral_top,mineral_bot,DATEYR,LATDD,LONDD
,String,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64?,Float64,Float64
1,AL0007,17.0032,0.0,0,160,-1,-1,0,160,1994,41.0694,19.9956
2,AL0008,6.79968,0.0,0,100,-1,-1,0,100,1994,40.215,20.6472
3,AL0009,10.19,0.0,0,97,-1,-1,0,97,1994,40.6372,20.7844
4,AL0010,16.182,0.0,0,110,-1,-1,0,110,1994,40.3453,19.4836
5,AL0011,19.5227,0.0,0,130,-1,-1,0,130,1994,41.3778,19.7703
6,AL0012,12.7518,0.0,0,173,-1,-1,0,173,1994,40.6553,20.9589
7,AL0013,14.3507,0.0,0,180,-1,-1,0,180,1994,40.5644,20.7325
8,AL0014,13.6746,0.0,0,130,-1,-1,0,130,1994,41.6928,20.4064
9,AL0015,23.7756,0.0,0,180,-1,-1,0,180,1994,41.37,19.7625


In [41]:
#doing the same with the clipped variant of the dataset
df_clipped_results = innerjoin(df_storage_clipped, df_clipped_depth_desig, df_site, on = :profile_id);

Results here:

(normal) https://drive.google.com/file/d/1S_TtUI-IyiXzoBQuEYTyPZQ3j-G5ku_4/view?usp=sharing

(clipped) https://drive.google.com/file/d/14tg2SseAl0EF9GQf8D4NZLCDmCmG__kd/view?usp=sharing